In [19]:
import datetime as dt

import numpy as np
from backtesting import Backtest
from backtesting.lib import Strategy

import algotrading_v40.data_accessors.cleaned as dac
import algotrading_v40.performance as perf
import algotrading_v40.structures.date_range as sdr
import algotrading_v40.structures.instrument_desc as sid

In [20]:
class PositionStrategy(Strategy):
  """
  Rebalance every bar so that the position equals the value
  in the `final_ba_position` column.
  """

  def init(self):
    # Nothing to pre-compute; we just need the method to exist
    super().init()

  def next(self):
    desired = int(self.data.final_ba_position[-1])  # target position
    current = self.position.size  # what we hold now
    delta = desired - current  # how much to adjust
    # print("T: ", len(self.data.final_ba_position))
    # print("desired: ", desired, "current: ", current, "delta: ", delta)
    if delta > 0:
      self.buy(size=delta)  # go longer / cover shorts
    elif delta < 0:
      self.sell(size=-delta)  # go shorter / reduce longs

In [21]:
data = dac.get_cleaned_data(
  [
    sid.EquityDesc(symbol="TRENT", market=sid.Market.INDIAN_MARKET),
    sid.EquityDesc(symbol="HDFCBANK", market=sid.Market.INDIAN_MARKET),
    sid.EquityDesc(symbol="GRASIM", market=sid.Market.INDIAN_MARKET),
    sid.EquityDesc(symbol="ICICIBANK", market=sid.Market.INDIAN_MARKET),
  ],
  sdr.DateRange(dt.date(2021, 1, 1), dt.date(2021, 4, 10)),
)
data._instrument_desc_to_df_list
df_icici = data.get_full_df_for_instrument_desc(
  sid.EquityDesc(symbol="ICICIBANK", market=sid.Market.INDIAN_MARKET)
)

In [22]:
# just testing the backtesting library
# so just creating a random signal'
# seed = np.random.randint(0, 1000000)
# print(f"Seed: {seed}")
np.random.seed(361159)
df_icici["final_ba_position"] = (
  (np.random.uniform(-50, 50, len(df_icici))).round().astype(int)
)

df_icici = df_icici.rename(
  columns={
    "open": "Open",
    "high": "High",
    "low": "Low",
    "close": "Close",
  }
)
df_icici = df_icici.iloc[46:248]
# df_icici = df_icici.iloc[46:247]
# we should start and end the backtest with 0 position
df_icici.loc[df_icici.index[-1], "final_ba_position"] = 0

In [23]:
df_icici

,Open,High,Low,Close,volume,final_ba_position
bar_close_timestamp,,,,,,
2021-01-01 04:31:59.999000+00:00,533.05,533.25,533.00,533.25,9782,-18
2021-01-01 04:32:59.999000+00:00,533.20,533.25,533.00,533.00,10696,-9
2021-01-01 04:33:59.999000+00:00,533.00,533.30,533.00,533.30,4401,-26
2021-01-01 04:34:59.999000+00:00,533.30,533.45,533.20,533.40,10959,26
2021-01-01 04:35:59.999000+00:00,533.30,533.50,533.25,533.35,4708,39
...,...,...,...,...,...,...
2021-01-01 07:48:59.999000+00:00,528.70,528.90,528.65,528.75,20460,17
2021-01-01 07:49:59.999000+00:00,528.75,528.85,528.75,528.85,14152,42
2021-01-01 07:50:59.999000+00:00,528.80,528.95,528.80,528.85,12239,17


In [24]:
# df_icici = df_icici.iloc[46:247] last 3
# 2021-01-01 07:49:59.999000+00:00	528.75	528.85	528.75	528.85	14152	42
# 2021-01-01 07:50:59.999000+00:00	528.80	528.95	528.80	528.85	12239	17
# 2021-01-01 07:51:59.999000+00:00	528.85	528.85	528.60	528.65	15994	0

# df_icici = df_icici.iloc[46:248] last 4
# 2021-01-01 07:49:59.999000+00:00	528.75	528.85	528.75	528.85	14152	42
# 2021-01-01 07:50:59.999000+00:00	528.80	528.95	528.80	528.85	12239	17
# 2021-01-01 07:51:59.999000+00:00	528.85	528.85	528.60	528.65	15994	-18
# 2021-01-01 07:52:59.999000+00:00	528.65	528.70	528.55	528.55	23922	0


In [25]:
np.round(perf.compute_backtesting_return(df_icici)[-1], 5)

np.float64(-3.23176)

In [26]:
bt = Backtest(
  df_icici,
  PositionStrategy,
  commission=0.0005,
  trade_on_close=True,
  hedging=False,
  exclusive_orders=False,
)
stats = bt.run()

Backtest.run:   0%|          | 0/201 [00:00<?, ?bar/s]

In [27]:
np.round(stats["Return [%]"], 5)

np.float64(-3.20218)